In [114]:
#q1
with open('/etc/os-release', 'r') as f:
    m=f.read()
m

'PRETTY_NAME="Ubuntu 24.04.2 LTS"\nNAME="Ubuntu"\nVERSION_ID="24.04"\nVERSION="24.04.2 LTS (Noble Numbat)"\nVERSION_CODENAME=noble\nID=ubuntu\nID_LIKE=debian\nHOME_URL="https://www.ubuntu.com/"\nSUPPORT_URL="https://help.ubuntu.com/"\nBUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"\nPRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"\nUBUNTU_CODENAME=noble\nLOGO=ubuntu-logo\n'

In [115]:
import subprocess

In [116]:
#q2
subprocess.run(['lscpu'], capture_output=True, text=True).stdout

'Architecture:                         x86_64\nCPU op-mode(s):                       32-bit, 64-bit\nAddress sizes:                        46 bits physical, 48 bits virtual\nByte Order:                           Little Endian\nCPU(s):                               2\nOn-line CPU(s) list:                  0,1\nVendor ID:                            GenuineIntel\nModel name:                           Intel(R) Xeon(R) CPU @ 2.20GHz\nCPU family:                           6\nModel:                                79\nThread(s) per core:                   2\nCore(s) per socket:                   1\nSocket(s):                            1\nStepping:                             0\nBogoMIPS:                             4399.99\nFlags:                                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pci

In [163]:
#q3
from pyarrow.fs import GcsFileSystem, FileSelector
fs = GcsFileSystem()
selector = FileSelector('shubham_10jain', recursive=False)
files = [f.path for f in fs.get_file_info(selector)]

files



['shubham_10jain/wi-schools-raw.parquet']

In [164]:
#q3
from pyarrow.fs import GcsFileSystem, FileSelector

fs = GcsFileSystem()
# List all files in the root of your bucket
selector = FileSelector('shubham_10jain', recursive=False)
files = [file_info.path for file_info in fs.get_file_info(selector)]
files

['shubham_10jain/wi-schools-raw.parquet']

In [165]:
#q4
file_info = fs.get_file_info('shubham_10jain/wi-schools-raw.parquet')
file_info.mtime_ns

1746221199210000000

In [166]:
# Dataform setup - Upload SQLX files
from google.cloud import dataform_v1beta1 as dataform

client = dataform.DataformClient()

PROJECT_ID = "solar-bolt-458603-v2"
REPOSITORY_ID = "Shubham_10jain"
WORKSPACE_ID = "Shubham_workspace"

workspace_path = f"projects/{PROJECT_ID}/locations/us-central1/repositories/{REPOSITORY_ID}/workspaces/{WORKSPACE_ID}"

wi_counties_sqlx = """
config {
  type: "table",
  name: "wi_counties",
  schema: "p8"
}

SELECT *
FROM 
  `bigquery-public-data.geo_us_boundaries.counties`
WHERE 
  state_fips_code = '55'
"""

with open("./definitions/wi_counties.sqlx", "w") as f:
    f.write(wi_counties_sqlx)
client.write_file(
    request={
        "workspace": workspace_path,
        "path": "definitions/wi_counties.sqlx",
        "contents": wi_counties_sqlx.encode()
    }
)
print("Uploaded wi_counties.sqlx")

Uploaded wi_counties.sqlx


In [167]:
# schools.sqlx
PROJECT_ID = "solar-bolt-458603-v2"
REPOSITORY_ID = "Shubham_10jain"
WORKSPACE_ID = "Shubham_workspace"


workspace_path = f"projects/{PROJECT_ID}/locations/us-central1/repositories/{REPOSITORY_ID}/workspaces/{WORKSPACE_ID}"

schools_sqlx = """
config {
    type: "operations",
    name: "schools"
    hasOutput: true
}
 
LOAD DATA OVERWRITE solar-bolt-458603-v2.p8.schools
FROM FILES (
    format = 'PARQUET',
    uris = ['gs://shubham_10jain/wi-schools-raw.parquet']
)
"""

with open("./definitions/schools.sqlx", "w") as f:
    f.write(schools_sqlx)

client.write_file(
    request=dataform.WriteFileRequest(
        workspace=workspace_path,
        path="definitions/schools.sqlx",
        contents=schools_sqlx.encode("utf-8")
    )
)


In [168]:
# wi_county_schools.sqlx

PROJECT_ID = "solar-bolt-458603-v2"
REPOSITORY_ID = "Shubham_10jain"
WORKSPACE_ID = "Shubham_workspace"


workspace_path = f"projects/{PROJECT_ID}/locations/us-central1/repositories/{REPOSITORY_ID}/workspaces/{WORKSPACE_ID}"

county_schools_sqlx = """
config {
    type: "table",
    name: "county_schools"
    schema: "p8"
}
 
SELECT
    s.*,
    ST_GEOGPOINT(s.longitude, s.latitude) AS location,
    c.county_name
FROM ${ref('schools')} s
JOIN ${ref('wi_counties')} c
ON ST_INTERSECTS(
    ST_GEOGPOINT(s.longitude, s.latitude),
    c.county_geom
)
"""
with open("./definitions/wi_county_schools.sqlx", "w") as f:
    f.write(county_schools_sqlx)
    
client.write_file(
    request=dataform.WriteFileRequest(
        workspace=workspace_path,
        path="definitions/wi_county_schools.sqlx",
        contents=county_schools_sqlx.encode("utf-8")
    )
)


In [169]:
from google.cloud import dataform_v1beta1
from google.api_core.exceptions import GoogleAPICallError\


PROJECT_ID = "solar-bolt-458603-v2"
REPOSITORY_ID = "Shubham_10jain"
WORKSPACE_ID = "Shubham_workspace"


workspace_path = f"projects/{PROJECT_ID}/locations/us-central1/repositories/{REPOSITORY_ID}/workspaces/{WORKSPACE_ID}"

compilation_result = dataform_v1beta1.types.CompilationResult(

    workspace=workspace_path,
    code_compilation_config=dataform_v1beta1.types.CodeCompilationConfig(
        default_database="solar-bolt-458603-v2",  # Your BigQuery project ID
        default_schema="p8"  # Dataset where the table will reside
    )
)
 

response = client.create_compilation_result(
    parent=f"projects/{PROJECT_ID}/locations/us-central1/repositories/{REPOSITORY_ID}",
    compilation_result=create_compilation_result

)

 

In [170]:
#q5
from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client()

# Query to count counties in Wisconsin
query = """
SELECT COUNT(*) AS county_count
FROM `solar-bolt-458603-v2.p8.wi_counties`
"""

# Execute the query (using .query() method as required)
result = client.query(query).to_dataframe()

# Extract the count value as a Python int
count = int(result.iloc[0]['county_count'])
count

72

In [171]:
#q6
query = """
SELECT COUNT(*) as school_count
FROM `solar-bolt-458603-v2.p8.wi_county_schools`
WHERE agency_type = 'Public school'
"""
result = bq.query(query).to_dataframe()
count=int(result.iloc[0]['school_count'])
count

2116

In [75]:
#q7
from google.cloud import dataform_v1beta1

client = dataform_v1beta1.DataformClient()
compilation_name = response.name

actions_response = client.query_compilation_result_actions(
    request={"name": compilation_name}
)

dependencies = {}
for action in actions_response:
    if action.target and action.relation:
        target_name = action.target.name.split('/')[-1]
        deps = [dep.name.split('/')[-1] for dep in action.relation.dependency_targets]
        dependencies[target_name] = deps

dependencies

{'first_view': [],
 'second_view': ['first_view'],
 'wi_counties': [],
 'wi_county_schools': ['schools', 'wi_counties']}

In [78]:
#q8
from google.cloud import bigquery

client = bigquery.Client()

query = """
SELECT 
  county_name,
  COUNT(*) as school_count
FROM 
  `solar-bolt-458603-v2.p8.wi_county_schools`
GROUP BY 
  county_name
ORDER BY 
  school_count DESC
"""


result = client.query(query).to_dataframe()

answer_q8 = {county: int(count) for county, count in zip(result['county_name'], result['school_count'])}
answer_q8

{'Milwaukee': 420,
 'Dane': 217,
 'Waukesha': 177,
 'Brown': 123,
 'Outagamie': 96,
 'Rock': 72,
 'Marathon': 72,
 'Racine': 68,
 'Winnebago': 65,
 'Sheboygan': 63,
 'La Crosse': 59,
 'Kenosha': 56,
 'Washington': 56,
 'Walworth': 56,
 'Fond du Lac': 55,
 'Eau Claire': 54,
 'Jefferson': 52,
 'Dodge': 51,
 'Wood': 47,
 'Manitowoc': 44,
 'Sauk': 42,
 'Columbia': 41,
 'Barron': 40,
 'Grant': 39,
 'Portage': 38,
 'Ozaukee': 38,
 'Clark': 38,
 'Waupaca': 36,
 'St. Croix': 35,
 'Chippewa': 35,
 'Calumet': 30,
 'Shawano': 29,
 'Pierce': 29,
 'Marinette': 29,
 'Vernon': 29,
 'Polk': 28,
 'Monroe': 28,
 'Trempealeau': 27,
 'Juneau': 26,
 'Green': 23,
 'Dunn': 22,
 'Oconto': 20,
 'Lafayette': 18,
 'Door': 18,
 'Douglas': 18,
 'Crawford': 17,
 'Oneida': 17,
 'Iowa': 16,
 'Lincoln': 16,
 'Washburn': 16,
 'Green Lake': 15,
 'Kewaunee': 15,
 'Richland': 14,
 'Langlade': 13,
 'Taylor': 13,
 'Vilas': 13,
 'Sawyer': 13,
 'Waushara': 13,
 'Ashland': 12,
 'Bayfield': 12,
 'Buffalo': 11,
 'Rusk': 11,
 'Ma

In [182]:
#q9
 
config = bigquery.QueryJobConfig(use_query_cache=False)

q = bq.query(query, job_config=config)
 
q.cache_hit
 
q.total_bytes_processed / 1024**2

q.total_bytes_billed / 1024**2
 
TB = 1024 ** 4

TB // q.total_bytes_billed



104857

In [183]:
#q10
from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client()

# Define the query to find closest public high schools for each public middle school in Dane County
query = """
WITH middle_schools AS (
    SELECT 
        school_name,
        location
    FROM `solar-bolt-458603-v2.p8.wi_county_schools`
    WHERE 
        county_name = 'Dane' AND
        agency_type = 'Public school' AND
        school_type = 'Middle School'
),

high_schools AS (
    SELECT 
        school_name,
        location
    FROM `solar-bolt-458603-v2.p8.wi_county_schools`
    WHERE 
        county_name = 'Dane' AND
        agency_type = 'Public school' AND
        school_type = 'High School'
)

SELECT
    ms.school_name AS middle_school,
    MIN_BY(hs.school_name, ST_DISTANCE(ms.location, hs.location)) AS closest_high_school
FROM middle_schools ms
CROSS JOIN high_schools hs
GROUP BY middle_school
"""

# Replace {YOUR_PROJECT_ID} with your actual project ID
query = query.format(YOUR_PROJECT_ID=client.project)

# Execute the query and convert to dictionary
query_job = client.query(query)
results = query_job.result()

# Convert to dictionary format
closest_high_schools = {row.middle_school: row.closest_high_school for row in results}

closest_high_schools

{'Kromrey Middle': 'Middleton High',
 'Prairie View Middle': 'Sun Prairie West High',
 'Belleville Middle': 'Belleville High',
 'Mount Horeb Middle': 'Mount Horeb High',
 'Glacier Creek Middle': 'Middleton High',
 'Marshall Middle': 'Marshall High',
 'James Wright Middle': 'West High',
 'River Bluff Middle': 'Stoughton High',
 'Black Hawk Middle': 'Shabazz-City High',
 'Indian Mound Middle': 'McFarland High',
 'Wisconsin Heights Middle': 'Wisconsin Heights High',
 'Waunakee Middle': 'Waunakee High',
 'Toki Middle': 'Vel Phillips Memorial High School',
 'De Forest Middle': 'De Forest High',
 'Central Heights Middle': 'Prairie Phoenix Academy',
 'Ezekiel Gillespie Middle School': 'Vel Phillips Memorial High School',
 'Oregon Middle': 'Oregon High',
 'Whitehorse Middle': 'Monona Grove High',
 'Cherokee Heights Middle': 'Capital High',
 'Glacial Drumlin School': 'LaFollette High',
 'Deerfield Middle': 'Deerfield High',
 "O'Keeffe Middle": 'Innovative High',
 'Patrick Marsh Middle': 'Prairi